In [1]:
#pip install git+git://github.com/PnS2019/pnslib.git

In [2]:
import rospy
from sensor_msgs.msg import Image
from cv_bridge import CvBridge
import cv2
from pnslib import utils

pub = rospy.Publisher("output_image",Image,queue_size=1)

In [3]:
face_cascade = cv2.CascadeClassifier(utils.get_haarcascade_path('haarcascade_frontalface_default.xml'))
eye_cascade = cv2.CascadeClassifier(utils.get_haarcascade_path('haarcascade_eye.xml'))
        
def process_image(msg):
    global pub
    try:
        bridge = CvBridge()
        img_rgb = bridge.imgmsg_to_cv2(msg,"rgb8")
        ##処理の内容##

        # search face
        gray = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)
        
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)
        
        for (x, y, w, h) in faces:
            cv2.rectangle(img_rgb, (x, y), (x+w, y+h), (255, 0, 0), 2)
            roi_gray = gray[y:y+h, x:x+w]
            roi_color = img_rgb[y:y+h, x:x+w]
            eyes = eye_cascade.detectMultiScale(roi_gray)
            for (ex, ey, ew, eh) in eyes:
                cv2.rectangle(roi_color, (ex, ey), (ex+ew, ey+eh), (0, 255, 0), 2)
        ###
        output_img = bridge.cv2_to_imgmsg(img_rgb,"rgb8")
        pub.publish(output_img)
    except Exception as err:
        print(err)

def start_node():
    rospy.init_node('face_detection')
    rospy.Subscriber("camera/color/image_raw",Image,process_image)
    rospy.spin()
    
#### main -----------------------------------------------------------
try:
    start_node()
except rospy.ROSInterruptException as err:
    print(err)